In [26]:
import numpy as np
from time import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd

%run Noise.ipynb
%run general_solver.ipynb

In [12]:
N = 1 # number of realizations of the noise and initial conditions

# time space boundaries
s, t, a, b = 0, 1, 0, 1

# time space mesh sizes
dt, dx = 1e-3, 0.005

# Generate N realizations of space time white noise. 
# Might take a while if the dt = 1e-4. Around 441 seconds to generate 1000 noises with mesh dt = 1e-4 on my laptop
# and 146 seconds with dt = 1e-3.
# Might be worth spliting into batches of 100 noises and apply general_1d_solver separately if dt = 1e-4. In general_1d_solver
# noise W is transformed into dW which might take a while if N is large and dt is small.

s_ = -time()
W = Noise().WN_space_time_many(s, t, dt, a, b, dx, N)
print(s_+time())

#space time domain
T, X = np.linspace(0,1, W.shape[1]), np.linspace(0,1, W.shape[2])

# generate N realizations of initial conditions like in the paper
u0 = np.array([[x*(1-x) for x in X[:-1]] for _ in range(N)])
lambd = 0.1 # strength of the randomness of the noise

u0 = u0 + lambd*Noise().initial(N, X, p = 10)[:,:-1] # Delete the last space point which is equal to the first one


0.3660271167755127


In [13]:
# Kuramoto–Sivashinsky SPDE.

L_ks = [0,0,-1,0,-1]
mu_ks = lambda x : 0
sigma_ks = lambda x : 1

KS, _, _ = general_1d_solver(L, u0, W[:,:,:-1], mu = mu_ks, sigma = sigma_ks, KPZ = 0.5)

# Ignore the last space point in the noise which is equal to the first one by periodicity in W[:,:,:-1]

100%|██████████| 1000/1000 [00:00<00:00, 2939.94it/s]


In [18]:
# Random Nonlinear Schrodinger equation (no noise)

L_s = [0,0,1j,0,0]
mu_s = lambda x : -1j*x*np.absolute(x)**2
sigma_s = lambda x : 0

# create imaginary part of the initial condition

u0_im = u0 + lambd*Noise().initial(N, X, p = 10)[:,:-1]

u0_s = u0 + 1j*u0_im

Schrodinger, _, _ = general_1d_solver(L_s, u0_s, W[:,:,:-1], mu = mu_s, sigma = sigma_s, compl = True)

100%|██████████| 1000/1000 [00:00<00:00, 1060.31it/s]


In [36]:
# Phi^4 equation with Dirichlet boundary conditions

u0_d = np.array([[x*(1-x) for x in X] for _ in range(N)])
u0_d = u0_d + lambd*Noise().initial(N, X, p = 10, Dirichlet = True)

mu_d = lambda x: -x**3
sigma_d = lambda x: x

solver = SPDE(Type = 'P', IC = u0_d, mu = mu_d, sigma = sigma_d, BC = 'D', T = T, X = X)

Phi4_D = solver.Parabolic(W)

100%|██████████| 1000/1000 [00:00<00:00, 6379.01it/s]
